## Transfer Learning VGG 16 and VGG 19 using Keras

Please download the dataset from the below url

In [15]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [26]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [27]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'


In [28]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [29]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [30]:
  # useful for getting number of output classes
folders = glob('Datasets/train/*')

In [31]:
len(folders)

4

In [32]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [33]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [34]:

# view the structure of the model
model.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [35]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [36]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [37]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [38]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 18 images belonging to 4 classes.


In [39]:
# fit the model
# Run the cell. It will take some time to execute

model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
61/61 [==============================] - 205s 3s/step - loss: 3.3726 - accuracy: 0.4131 - val_loss: 1.2988 - val_accuracy: 0.5556
Epoch 2/20
61/61 [==============================] - 192s 3s/step - loss: 0.9817 - accuracy: 0.6007 - val_loss: 0.9628 - val_accuracy: 0.6111
Epoch 3/20
61/61 [==============================] - 165s 3s/step - loss: 0.8829 - accuracy: 0.6427 - val_loss: 0.8675 - val_accuracy: 0.6667
Epoch 4/20
61/61 [==============================] - 146s 2s/step - loss: 0.9868 - accuracy: 0.6151 - val_loss: 0.9829 - val_accuracy: 0.6667
Epoch 5/20
61/61 [==============================] - 146s 2s/step - loss: 0.9474 - accuracy: 0.6597 - val_loss: 1.1753 - val_accuracy: 0.6667
Epoch 6/20
61/61 [==============================] - 146s 2s/step - loss: 0.8711 - accuracy: 0.6643 - val_loss: 0.9434 - val_accuracy: 0.6667
Epoch 7/20
61/61 [==============================] - 148s 2s/step - loss: 0.8237 - accuracy: 0.6735 - val_loss: 1.3774 - val_accuracy: 0.5556
Epoch 8/20
61

In [40]:
import matplotlib.pyplot as plt

In [41]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: name 'r' is not defined

In [42]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')

In [43]:

y_pred = model.predict(test_set)


In [44]:
y_pred

array([[5.9446055e-02, 3.5755402e-01, 8.2737217e-03, 5.7472628e-01],
       [1.2009325e-03, 7.0976865e-01, 7.2105511e-05, 2.8895825e-01],
       [1.3426662e-03, 8.5080343e-01, 2.0236384e-02, 1.2761752e-01],
       [2.2449721e-02, 1.0689752e-05, 1.3014236e-03, 9.7623819e-01],
       [5.0827570e-04, 5.7950724e-02, 6.9752978e-03, 9.3456578e-01],
       [7.5539353e-04, 4.2490847e-04, 9.9131167e-01, 7.5080316e-03],
       [1.8677429e-03, 6.2046260e-01, 1.0643484e-02, 3.6702621e-01],
       [9.0926783e-03, 9.5323304e-04, 9.8717982e-01, 2.7741999e-03],
       [4.3301177e-01, 2.7945957e-01, 8.8201808e-03, 2.7870849e-01],
       [9.9977547e-01, 3.6861792e-08, 1.3550407e-04, 8.9086017e-05],
       [1.0719353e-03, 1.1526137e-06, 9.9891305e-01, 1.3826518e-05],
       [6.5561435e-05, 1.5905113e-06, 9.9673009e-01, 3.2027599e-03],
       [1.3958977e-03, 9.6345985e-01, 4.9311588e-05, 3.5094891e-02],
       [3.6005486e-02, 5.6955969e-01, 1.7245094e-01, 2.2198388e-01],
       [8.4897252e-03, 2.1458471e-

In [45]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [46]:
y_pred

array([3, 1, 1, 3, 3, 2, 1, 2, 0, 0, 2, 2, 1, 1, 2, 3, 3, 3], dtype=int64)

In [47]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [48]:
model=load_model('model_resnet50.h5')

In [49]:
img_data

NameError: name 'img_data' is not defined

In [51]:
img=image.load_img('Datasets/Test/fresh cotton leaf/d(341).jpg',target_size=(224,224))



FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/Test/fresh cotton leaf/d(341).jpg'

In [12]:
x=image.img_to_array(img)
x

array([[[254., 254., 254.],
        [254., 254., 254.],
        [254., 254., 254.],
        ...,
        [254., 254., 254.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[254., 254., 254.],
        [254., 254., 254.],
        [254., 254., 254.],
        ...,
        [254., 254., 254.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[254., 254., 254.],
        [254., 254., 254.],
        [254., 254., 254.],
        ...,
        [254., 254., 254.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [13]:
x.shape

(224, 224, 3)

In [14]:
x=x/255

In [15]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [16]:
model.predict(img_data)

array([[0.9745471, 0.0254529]], dtype=float32)

In [17]:
a=np.argmax(model.predict(img_data), axis=1)

In [102]:
a==1

array([ True])

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.3.0'